In [1]:
import datetime
import pandas as pd
import numpy as np
import os
import optuna
from os.path import curdir
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data_path = ('train.csv')
df = pd.read_csv(data_path)

In [3]:
df['EJ'] = df['EJ'].replace('A', '1')

In [4]:
df['EJ'] = df['EJ'].replace('B', '2')

In [5]:
df['EJ'] = df['EJ'].astype('float64')

In [6]:
# Calculate the mean for "BQ" column and fill missing values with the mean
median_bq = df["BQ"].mean()
df["BQ"].fillna(median_bq, inplace=True)


median_cb = df["CB"].mean()
df["CB"].fillna(median_cb, inplace=True)


median_cc = df["CC"].mean()
df["CC"].fillna(median_cc, inplace=True)

median_DU = df["DU"].mean()
df['DU'].fillna(median_DU, inplace=True)

median_EL = df["EL"].mean()
df["EL"].fillna(median_EL, inplace=True)

median_FC = df["FC"].mean()
df["FC"].fillna(median_FC, inplace=True)

median_FL = df["FL"].mean()
df["FL"].fillna(median_FL, inplace=True)

median_FS = df["FS"].mean()
df["FS"].fillna(median_FS, inplace=True)

median_GL = df["GL"].mean()
df["GL"].fillna(median_GL, inplace=True)

In [7]:
df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [8]:
df=df.drop(['Id'], axis=1)

In [9]:
df

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,5654.07556,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.87769,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,4517.86560,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [10]:
#Normalize the Data

X = df.copy()
scaler = StandardScaler()
features = ['AB','AF','AH','AM','AR','AX', 'AY','AZ','BC','BD ','BN','BP','BQ','BR','BZ','CB','CC','CD ','CF','CH','CL','CR','CS','CU','CW ','DA','DE','DF','DH','DI','DL',
 'DN','DU','DV','DY','EB','EE','EG','EH','EJ','EL','EP','EU','FC','FD ','FE','FI','FL','FR','FS','GB','GE','GF','GH','GI','GL',]
X[features] = scaler.fit_transform(X[features])


In [11]:
X.shape
X.info()
X.describe().transpose()
X.head(10)
X.tail()

(617, 57)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 57 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AB      617 non-null    float64
 1   AF      617 non-null    float64
 2   AH      617 non-null    float64
 3   AM      617 non-null    float64
 4   AR      617 non-null    float64
 5   AX      617 non-null    float64
 6   AY      617 non-null    float64
 7   AZ      617 non-null    float64
 8   BC      617 non-null    float64
 9   BD      617 non-null    float64
 10  BN      617 non-null    float64
 11  BP      617 non-null    float64
 12  BQ      617 non-null    float64
 13  BR      617 non-null    float64
 14  BZ      617 non-null    float64
 15  CB      617 non-null    float64
 16  CC      617 non-null    float64
 17  CD      617 non-null    float64
 18  CF      617 non-null    float64
 19  CH      617 non-null    float64
 20  CL      617 non-null    float64
 21  CR      617 non-null    float64
 22  CS

,count,mean,std,min,25%,50%,75%,max
AB,617.0,-3.157365e-17,1.000811,-0.846058,-0.480851,-2.617267e-01,0.176522,12.146180
AF,617.0,-2.218647e-16,1.000811,-1.439844,-0.567627,-1.660654e-01,0.374000,10.957858
AH,617.0,-1.504289e-16,1.000811,-0.261669,-0.261669,-2.616690e-01,-0.038243,14.025086
AM,617.0,1.871365e-17,1.000811,-0.513710,-0.383201,-2.646045e-01,0.002459,8.490531
AR,617.0,-4.759384e-17,1.000811,-0.189295,-0.189295,-1.892948e-01,-0.189295,16.061825
AX,617.0,-3.753525e-16,1.000811,-1.900558,-0.555878,-2.014664e-01,0.347525,12.835330
AY,617.0,-1.862368e-16,1.000811,-0.083417,-0.083417,-8.341747e-02,-0.056366,24.624368
AZ,617.0,-8.097251e-17,1.000811,-1.649292,-0.560571,-2.418314e-02,0.552796,6.534241
BC,617.0,-2.868374e-17,1.000811,-0.104787,-0.104787,-1.047870e-01,-0.045639,22.355222
BD,617.0,-5.542118e-17,1.000811,-1.211299,-0.395738,-1.167414e-01,0.227070,15.803958


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,0.162487,-0.035806,-0.250869,-0.940094,-0.410260,-0.655511,-0.948991,0.531241,-0.815752,1
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.458281,-0.060566,0.113218,-1.145070,-0.410260,0.687893,-0.238862,-0.509218,1.304239,0
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,0.198331,-0.051023,0.596934,1.637944,-0.299210,-0.051850,-0.351743,-0.424754,-0.808323,0
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,0.060027,-0.060566,-0.105234,-0.219883,-0.342195,-0.650833,0.858232,1.101332,-0.812311,0
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.236971,0.896815,-0.230064,-0.432313,0.099920,-0.318309,1.409422,-0.395228,-0.818054,1
5,-0.572153,-0.385562,-0.261669,-0.436720,-0.189295,-0.601048,-0.083417,0.455666,-0.104787,-0.037381,...,-0.458281,-0.060566,0.570928,0.119259,-0.410260,0.491176,-0.300777,0.881515,1.304239,0
6,-0.275422,-0.769368,-0.261669,-0.439076,0.493251,-1.424535,-0.083417,-0.924444,-0.104787,0.119274,...,-0.089279,-0.053302,0.035199,-0.285103,0.421603,-0.586801,0.271554,1.366078,-0.821111,0
7,-0.444330,-1.103156,-0.261669,-0.255403,-0.189295,-0.218839,-0.083417,-0.266282,-0.104787,-0.102558,...,0.100953,-0.060566,-0.110436,-0.228269,-0.152855,0.346076,0.377262,-0.528736,-0.818417,0
8,-0.279987,-0.114675,-0.261669,-0.144675,-0.189295,-0.597574,-0.083417,-0.534476,-0.067982,0.403786,...,-0.058454,-0.046893,-0.271674,-0.348459,0.107616,-0.521789,-0.174305,0.112900,-0.820304,0
9,-0.325638,0.733826,-0.261669,-0.373177,-0.189295,-0.373460,-0.083417,0.027281,-0.104787,-0.132066,...,-0.458281,-0.039137,-0.079228,0.379671,-0.227573,-0.471445,-0.963337,-0.476958,1.304239,0


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
612,-0.699975,-0.161828,0.040232,-0.422762,0.275215,-0.802577,0.040875,-0.464891,-0.080610,-0.395082,...,-0.458281,-0.045332,-0.271674,-1.177680,0.593022,-0.340465,-0.694915,0.513497,1.304239,0
613,-0.088253,0.852755,-0.261669,0.108831,0.556117,0.170319,-0.082686,0.473063,-0.065661,0.100596,...,0.417330,-0.045702,0.003992,1.519617,2.535523,-0.599582,-0.186764,2.048314,-0.813328,0
614,-0.106514,-0.453742,0.090140,0.235206,-0.011673,0.990330,-0.083417,1.113828,-0.104787,0.178374,...,-0.458281,-0.060566,-0.271674,-0.076400,-0.019561,-0.424307,-0.540129,1.903449,1.304239,0
615,-0.243466,-0.973904,-0.261669,-0.219353,-0.189295,0.955584,-0.083417,-0.699741,-0.104787,-0.275772,...,0.333153,-0.054771,0.191237,0.387591,-0.410260,-0.657500,-0.646592,-0.370599,-0.809518,0
616,0.012178,-0.360885,3.350987,1.048310,-0.189295,-0.920714,0.135921,-1.649292,-0.001602,-0.838845,...,-0.458281,-0.047645,-0.209259,-0.706235,-0.410260,-0.404872,1.446419,1.773276,1.304239,0


In [12]:
df

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,5654.07556,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.87769,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,4517.86560,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [13]:
from sklearn.model_selection import train_test_split
y=df['Class']
X=df.drop(columns=['Class'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [14]:
#Decision trees

In [15]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=0)

scores = cross_val_score(clf, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.8688


In [16]:
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=5, random_state=0)

scores = cross_val_score(clf2, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.8656


In [17]:
clf3 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=6, random_state=0)

scores = cross_val_score(clf3, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.8640


In [18]:
#Hyperparameter Tuning

In [19]:
#Grid Search

In [20]:
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier(random_state=42)

params = {'criterion': ('gini', 'entropy'), 
          'splitter': ('best', 'random'), 
          'class_weight': ('balanced', None), 
          'max_depth': [2, 5, 10, 20], 
          'min_samples_leaf': [1, 5, 10],
          'max_features':[0.25, 0.5, 0.75, 1.0]}

search = GridSearchCV(clf, params, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

Fitting 10 folds for each of 384 candidates, totalling 3840 fits


In [21]:
search.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 0.5,
 'min_samples_leaf': 5,
 'splitter': 'best'}

In [22]:
search.best_score_

0.8095472939155084

In [23]:
#Random Search

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

clf = DecisionTreeClassifier(random_state=42)

params = {"criterion": ["gini", "entropy"],
          "splitter": ["best", "random"],
          "class_weight": ['balanced', None], 
          "max_depth": randint(2, 21),
          "min_samples_leaf": randint(1, 11),
          "max_features": uniform(0.0, 1.0)}

search = RandomizedSearchCV(clf, param_distributions=params, n_iter=1000, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


In [71]:
search.best_params_
search.best_score_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 13,
 'max_features': 0.9618021606691484,
 'min_samples_leaf': 5,
 'splitter': 'best'}

0.8137206069464735

In [72]:
#Halving Grid Search

In [73]:
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV

clf = DecisionTreeClassifier(random_state=42)

params = {'criterion': ('gini', 'entropy'), 
          'splitter': ('best', 'random'), 
          'class_weight': ('balanced', None), 
          'max_depth': [2, 5, 10, 20], 
          'min_samples_leaf': [1, 5, 10],
          'max_features':[0.25, 0.5, 0.75, 1.0]}

search = HalvingGridSearchCV(clf, params, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

n_iterations: 3
n_required_iterations: 6
n_possible_iterations: 3
min_resources_: 40
max_resources_: 617
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 384
n_resources: 40
Fitting 10 folds for each of 384 candidates, totalling 3840 fits
----------
iter: 1
n_candidates: 128
n_resources: 120
Fitting 10 folds for each of 128 candidates, totalling 1280 fits
----------
iter: 2
n_candidates: 43
n_resources: 360
Fitting 10 folds for each of 43 candidates, totalling 430 fits


In [74]:
search.best_params_
search.best_score_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 0.75,
 'min_samples_leaf': 5,
 'splitter': 'best'}

0.788410677406537

In [ ]:
#OPTUNA

In [76]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 1.3 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [78]:
def objective_dt(trial, X, y):

  # Now, define all the hyperparams we want to vary, and what values they are allowed
  # to take.
  #
  # Each trial, optuna will automatically choose values for each hyperparam.
  hyper_params = {
        
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),

        "max_depth": trial.suggest_int("max_depth", 5, 100, step=5),

        "min_samples_split": trial.suggest_int("min_samples_split", 5, 100, step=5),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 5, 100, step=5),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10, 1010, step=100),

        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),

        "random_state": 77,
  }
    

  # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
  clf = DecisionTreeClassifier(**hyper_params)

  # Run CV to see how well these hyper_params do
  cv_scores = cross_val_score(clf, X, y, cv=10, scoring="f1_macro")
  score = np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
  return score
     

In [79]:
study = optuna.create_study(direction="maximize")


[I 2023-07-13 11:20:53,720] A new study created in memory with name: no-name-014878af-a263-4d74-b2a1-9a6d6685b0f7


In [80]:
study.optimize(lambda trial: objective_dt(trial, X, y), n_trials=100,  gc_after_trial=True)

[I 2023-07-13 11:21:04,069] Trial 0 finished with value: 0.7319607567298598 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 25, 'min_samples_split': 40, 'min_samples_leaf': 90, 'max_leaf_nodes': 410, 'class_weight': None}. Best is trial 0 with value: 0.7319607567298598.
[I 2023-07-13 11:21:04,356] Trial 1 finished with value: 0.4520456771562967 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 30, 'min_samples_split': 95, 'min_samples_leaf': 65, 'max_leaf_nodes': 710, 'class_weight': None}. Best is trial 0 with value: 0.7319607567298598.
[I 2023-07-13 11:21:04,694] Trial 2 finished with value: 0.5729267734486392 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 80, 'min_samples_split': 15, 'min_samples_leaf': 70, 'max_leaf_nodes': 810, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7319607567298598.
[I 2023-07-13 11:21:05,166] Trial 3 finished with value: 0.7495588375358551 and parameters: {'criterio

[I 2023-07-13 11:21:48,575] Trial 28 finished with value: 0.7738120249349085 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 85, 'min_samples_split': 100, 'min_samples_leaf': 70, 'max_leaf_nodes': 910, 'class_weight': 'balanced'}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:21:50,054] Trial 29 finished with value: 0.7456724071178245 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 85, 'min_samples_split': 100, 'min_samples_leaf': 80, 'max_leaf_nodes': 1010, 'class_weight': None}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:21:50,851] Trial 30 finished with value: 0.6831666632284614 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 100, 'min_samples_split': 85, 'min_samples_leaf': 100, 'max_leaf_nodes': 910, 'class_weight': 'balanced'}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:21:51,979] Trial 31 finished with value: 0.7738120249349085 and paramete

[I 2023-07-13 11:22:12,079] Trial 55 finished with value: 0.7738120249349085 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 25, 'min_samples_split': 95, 'min_samples_leaf': 70, 'max_leaf_nodes': 710, 'class_weight': 'balanced'}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:22:12,881] Trial 56 finished with value: 0.7427617229066521 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 75, 'min_samples_split': 90, 'min_samples_leaf': 80, 'max_leaf_nodes': 810, 'class_weight': 'balanced'}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:22:13,398] Trial 57 finished with value: 0.5961708641305588 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 60, 'min_samples_split': 100, 'min_samples_leaf': 95, 'max_leaf_nodes': 810, 'class_weight': 'balanced'}. Best is trial 25 with value: 0.7738120249349085.
[I 2023-07-13 11:22:14,121] Trial 58 finished with value: 0.7585561732424597 and p

[I 2023-07-13 11:22:33,107] Trial 83 finished with value: 0.7575444326965864 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 90, 'min_samples_split': 15, 'min_samples_leaf': 40, 'max_leaf_nodes': 910, 'class_weight': 'balanced'}. Best is trial 59 with value: 0.7976009728009432.
[I 2023-07-13 11:22:33,901] Trial 84 finished with value: 0.7976009728009432 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 95, 'min_samples_split': 5, 'min_samples_leaf': 50, 'max_leaf_nodes': 1010, 'class_weight': 'balanced'}. Best is trial 59 with value: 0.7976009728009432.
[I 2023-07-13 11:22:34,651] Trial 85 finished with value: 0.7976009728009432 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 60, 'max_leaf_nodes': 910, 'class_weight': 'balanced'}. Best is trial 59 with value: 0.7976009728009432.
[I 2023-07-13 11:22:35,473] Trial 86 finished with value: 0.5813879922457811 and parameters: {

In [81]:
search.best_params_
search.best_score_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 0.75,
 'min_samples_leaf': 5,
 'splitter': 'best'}

0.788410677406537